In [27]:
import pandas
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
from astropy.nddata import CCDData
from pathlib import Path
from glowing_waffles.differential_photometry import *
from astrowidgets import ImageWidget, __version__
from glowing_waffles.photometry import *
import numpy as np
from comparison_functions import *
import astrowidgets
from seeing_profile_functions import *

In [28]:
print(astrowidgets.__version__)

0.1.0.dev162


In [29]:
directory_with_images = '/Volumes/My Passport for Mac/Strong 2019/all-18-7-23/2018-07-23-reduced'
#This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'kelt-16-S001-R001-C052-r.fit'
object_of_interest = 'kelt-16b'

In [30]:
df, RD, ra, dec, RD['coords'] = read_file('gaia_stars_1560877263.radec')

In [31]:
ccd, ccdx, ccdy, vsx, ra, dec, vsx['coords'], vsx_names = set_up('kelt-16b', directory_with_images, sample_image_for_finding_stars)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]
Private attributes "_naxis1" and "_naxis2" have been deprecated since v3.1.
Instead use the "pixel_shape" property which returns a list of NAXISj keyword values.
 [astropy.wcs.wcs]


In [32]:
apass, v_angle, RD_angle = match(ccd, RD, vsx)

In [33]:
Cmag = 11.380999999999998

In [34]:
apass_good_coord, good_stars = mag_scale(Cmag, apass, v_angle, RD_angle)

In [35]:
ent = in_field(apass_good_coord, ccd, ccdx, ccdy, apass, good_stars)

In [36]:
iw = ImageWidget()
iw

ImageWidget(children=(Image(value=b'', format='jpeg', height='500', layout="Layout(margin='0')", width='500'),…

In [37]:
make_markers(iw, directory_with_images, sample_image_for_finding_stars, RD, vsx, ent, 'kelt-16b')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [49]:
pic_name = 'kelt16-apertures-x'

In [50]:
iw.save(pic_name + '.png')

In [40]:
all_table = iw.get_markers(marker_name='all')

In [41]:
f = all_table['marker name'] == 'elim'

In [42]:
good_table = all_table[~f]

In [43]:
all_table = iw.get_markers(marker_name = 'all')
elims = np.array([name.startswith('elim') for name in all_table['marker name']])
elim_table = all_table[elims]
comp_table = all_table[~elims]

In [44]:
index, d2d, d3d = elim_table['coord'].match_to_catalog_sky(comp_table['coord'])
comp_table.remove_rows(index)

In [45]:
from ipywidgets import Output

out = Output()

In [46]:
wrap(iw, out)

<function comparison_functions.wrap.<locals>.cb(viewer, event, data_x, data_y)>

In [47]:
bind_map = iw._viewer.get_bindmap()
gvc = iw._viewer.get_canvas()
bind_map.map_event(None, ('shift',), 'ms_left', 'cursor')
gvc.add_callback('cursor-down', wrap(iw, out))

In [48]:
out

Output()

In [26]:
iw.stop_marking()